In [1]:
#  Importando Bibliotecas
import os
from google.cloud import bigquery
from dotenv import load_dotenv
import pandas as pd

In [2]:
# Carrega variáveis do .env
load_dotenv()

credencial = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
project_id = os.getenv("PROJECT_ID")
table_id_ = os.getenv("TABLE_DATE_ATT")
table_stg = os.getenv("TABLE_STG")


In [3]:
# Inicializa o cliente do BigQuery usando credenciais de serviço
# O parâmetro 'project_id' especifica o projeto GCP onde as queries serão executadas

client = bigquery.Client.from_service_account_json(credencial, project=project_id)


query = f"""
SELECT *
FROM `{table_stg}`
"""


In [4]:
# Executa e converte pra DataFrame
resultado = client.query(query)
df = resultado.to_dataframe()

/home/danielpedro/anaconda3/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [5]:
# Mostra as primeiras 5 linhas da tabela de datas
df.head()

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,3777.0,1678.0,pepperoni_s,1.0,2015-01-29,13:06:51,9.75,9.75,S,Classic,"Mozzarella Cheese, Pepperoni",The Pepperoni Pizza
1,6746.0,2976.0,pepperoni_s,1.0,2015-02-19,15:29:10,9.75,9.75,S,Classic,"Mozzarella Cheese, Pepperoni",The Pepperoni Pizza
2,7800.0,3430.0,pepperoni_s,1.0,2015-02-27,16:25:26.999999,9.75,9.75,S,Classic,"Mozzarella Cheese, Pepperoni",The Pepperoni Pizza
3,18660.0,8198.0,pepperoni_s,1.0,2015-05-17,18:03:19,9.75,9.75,S,Classic,"Mozzarella Cheese, Pepperoni",The Pepperoni Pizza
4,19687.0,8656.0,pepperoni_s,1.0,2015-05-25,15:16:37,9.75,9.75,S,Classic,"Mozzarella Cheese, Pepperoni",The Pepperoni Pizza


In [6]:
# Exibe as informações sobre o dataframe principal
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pizza_id           48620 non-null  float64
 1   order_id           48620 non-null  float64
 2   pizza_name_id      48620 non-null  object 
 3   quantity           48620 non-null  float64
 4   order_date         48620 non-null  dbdate 
 5   order_time         48620 non-null  dbtime 
 6   unit_price         48620 non-null  float64
 7   total_price        48620 non-null  float64
 8   pizza_size         48620 non-null  object 
 9   pizza_category     48620 non-null  object 
 10  pizza_ingredients  48620 non-null  object 
 11  pizza_name         48620 non-null  object 
dtypes: dbdate(1), dbtime(1), float64(5), object(5)
memory usage: 4.5+ MB


In [7]:
# Criando um novo DataFrame só com a coluna de datas dos pedidos
df_date = df[["order_date"]]

In [8]:
# Convertendo as datas para formato datetime, erros viram NaN
df_date['order_date'].apply(pd.to_datetime, errors='coerce')

0       2015-01-29
1       2015-02-19
2       2015-02-27
3       2015-05-17
4       2015-05-25
           ...    
48615   2015-10-04
48616   2015-07-22
48617   2015-12-01
48618   2015-10-06
48619   2015-11-19
Name: order_date, Length: 48620, dtype: datetime64[ns]

In [9]:
# Pega a data mais antiga e mais recente dos pedidos
data_pedido_max = df_date['order_date'].max()
data_pedido_min = df_date['order_date'].min()

print(data_pedido_min," - ",data_pedido_max)

2015-01-01  -  2015-12-31


In [10]:
# Criando o datagrame calendario utilizando o range do data max e data min 
df_date = pd.DataFrame({
    'date': pd.date_range(start=data_pedido_min, end=data_pedido_max, freq='D')
})

In [11]:
# Cria chaves numéricas para datas
df_date['sk_date'] = df_date['date'].dt.strftime('%Y%m%d').astype(int) 
df_date['date_id'] = df_date['date'].dt.strftime('%Y%m%d').astype(int)

# Extrai partes da data
df_date['year'] = df_date['date'].dt.year
df_date['month'] = df_date['date'].dt.month
df_date['day'] = df_date['date'].dt.day

# Dia da semana em números (Domingo=1, Segunda=2, ..., Sábado=7)
df_date['weekday_number'] = df_date['date'].dt.dayofweek + 1

# Nomes dos dias e meses
df_date['weekday_name'] = df_date['date'].dt.day_name(locale='en_US')
df_date['month_name'] = df_date['date'].dt.month_name(locale='en_US')

# Informações de semana, trimestre e semestre
df_date['week_of_year'] = df_date['date'].dt.isocalendar().week.astype(int)
df_date['quarter'] = df_date['date'].dt.quarter
df_date['semester'] = ((df_date['date'].dt.month - 1) // 6) + 1

# Períodos combinados
df_date['year_month'] = df_date['date'].dt.to_period('M').astype(str)
df_date['year_quarter'] = df_date['year'].astype(str) + '-Q' + df_date['quarter'].astype(str)
df_date['year_semester'] = df_date['year'].astype(str) + '-S' + df_date['semester'].astype(str)

# Flags e contagens
df_date['is_weekend'] = df_date['date'].dt.dayofweek >= 5
df_date['day_of_year'] = df_date['date'].dt.dayofyear

# Ordena por data e reseta índice
df_date = df_date.sort_values('date').reset_index(drop=True)

In [12]:
# Remove a hora da data, fica só ano-mês-dia
df_date['date'] = df_date['date'].dt.date

In [13]:
# Mostra as primeiras 5 linhas da tabela de datas
df_date.head()

,date,sk_date,date_id,year,month,day,weekday_number,weekday_name,month_name,week_of_year,quarter,semester,year_month,year_quarter,year_semester,is_weekend,day_of_year
0,2015-01-01,20150101,20150101,2015,1,1,4,Thursday,January,1,1,1,2015-01,2015-Q1,2015-S1,False,1
1,2015-01-02,20150102,20150102,2015,1,2,5,Friday,January,1,1,1,2015-01,2015-Q1,2015-S1,False,2
2,2015-01-03,20150103,20150103,2015,1,3,6,Saturday,January,1,1,1,2015-01,2015-Q1,2015-S1,True,3
3,2015-01-04,20150104,20150104,2015,1,4,7,Sunday,January,1,1,1,2015-01,2015-Q1,2015-S1,True,4
4,2015-01-05,20150105,20150105,2015,1,5,1,Monday,January,2,1,1,2015-01,2015-Q1,2015-S1,False,5


In [14]:
# Organizando as colunas na ordem desejada
df_date = df_date[['sk_date','date_id','date','year','month','day','weekday_name','weekday_number','month_name','week_of_year', 
                            'quarter','semester','year_month','year_quarter','year_semester', 
                            'is_weekend','day_of_year']]
 

In [15]:
# Mostra as primeiras 5 linhas da tabela de datas
df_date.head()

,sk_date,date_id,date,year,month,day,weekday_name,weekday_number,month_name,week_of_year,quarter,semester,year_month,year_quarter,year_semester,is_weekend,day_of_year
0,20150101,20150101,2015-01-01,2015,1,1,Thursday,4,January,1,1,1,2015-01,2015-Q1,2015-S1,False,1
1,20150102,20150102,2015-01-02,2015,1,2,Friday,5,January,1,1,1,2015-01,2015-Q1,2015-S1,False,2
2,20150103,20150103,2015-01-03,2015,1,3,Saturday,6,January,1,1,1,2015-01,2015-Q1,2015-S1,True,3
3,20150104,20150104,2015-01-04,2015,1,4,Sunday,7,January,1,1,1,2015-01,2015-Q1,2015-S1,True,4
4,20150105,20150105,2015-01-05,2015,1,5,Monday,1,January,2,1,1,2015-01,2015-Q1,2015-S1,False,5


In [16]:
# Verifica quantas datas são duplicadas e quantas são únicas
df_date.duplicated().value_counts()

False    365
Name: count, dtype: int64

In [17]:
# Configura para substituir a tabela existente
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
)

# Envia os dados pro BigQuery
job = client.load_table_from_dataframe(
    df_date,
    table_id_,
    job_config=job_config
)
job.result()  # espera acabar de carregar

print("dim_date carregada com sucesso!")


dim_date carregada com sucesso!


In [18]:
# Conta quantos registros tem na tabela carregada

query_check = f"SELECT COUNT(*) AS total FROM `{table_id_}`"
df_check = client.query(query_check).to_dataframe()
print(df_check)


/home/danielpedro/anaconda3/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   total
0    365
